In [17]:
import numpy as np
import pandas as pd
import ipywidgets as w
import bqplot.pyplot as bplt
from afterthought.portfolio import Portfolio

In [51]:
price = pd.read_csv('data/test_price.csv', index_col='date')
price.index = pd.to_datetime(price.index)
capital_base = 1000000
date_start= '2013-11-18'
rebalance_frequency = 50

# What Portfolio does

1. For each portfolio, we have many stocks
2. For each stock, its price is updated everyday
3. Since stock prices change, their weights change as well. (we say weight referring to the proportion of each stock's value vs our portfolio's total value.
4. What our Portfolio class do is as such: every, we input a new line of prices for each stock, then the Porfolio class will automatically calculated the new weights, returns, etc. with respect to the price change of stocks

In [58]:
def test_init():
    port = Portfolio(capital_base, date_start, price.columns, 'test')
    return port

def test_update():
    port = test_init()
    # pylint: disable=E1127
    idx = price.loc[date_start:].index
    day_counter = rebalance_frequency

    for t in idx:
        # receive newest price_df
        price_t = price.loc[t]    
        port.receive_price(price_t)
        
        # scheduled rebalance
        if day_counter == rebalance_frequency:
            day_counter = 0 # reset
            order = price_t.notna() / price_t.notna().sum()
            port.rebalance('to_weights', order)
        else:
            day_counter += 1
            port.keep_position()
    return port

# def test_output():
#     port = test_update()
#     port.summary
#     port.plot_performance()
#     port.plot_component_prices()
#     port.plot_position_values()
#     port.plot_postions()
#     port.plot_weights()

In [59]:
port = test_update()

In [62]:
port.weights_end[100:]

,DXCM,PODD,HLF,RMD,DVA
2014-04-14,0.185924,0.206778,0.160650,0.225746,0.220902
2014-04-15,0.179680,0.204449,0.163613,0.229795,0.222463
2014-04-16,0.200000,0.200000,0.200000,0.200000,0.200000
2014-04-17,0.200674,0.201925,0.197049,0.201696,0.198657
2014-04-21,0.197285,0.199194,0.204304,0.200839,0.198377
...,...,...,...,...,...
2015-11-11,0.169269,0.229221,0.185724,0.223271,0.192515
2015-11-12,0.171055,0.222003,0.188497,0.225312,0.193133
2015-11-13,0.173009,0.223613,0.187212,0.224948,0.191217
2015-11-16,0.168481,0.226346,0.190111,0.223236,0.191825


# Test bqplot and ipywidgets

In [81]:
data = port.weights_end[:20]

In [168]:
data

,DXCM,PODD,HLF,RMD,DVA
2013-11-18,0.200000,0.200000,0.200000,0.200000,0.200000
2013-11-19,0.205050,0.198943,0.197103,0.201411,0.197492
2013-11-20,0.202669,0.196889,0.207037,0.200107,0.193298
2013-11-21,0.206950,0.200513,0.202477,0.199180,0.190881
2013-11-22,0.202899,0.200348,0.210194,0.198500,0.188059
2013-11-25,0.200217,0.198816,0.204294,0.194593,0.202079
2013-11-26,0.202836,0.202126,0.202487,0.191861,0.200691
2013-11-27,0.206808,0.204849,0.202906,0.187738,0.197699
2013-11-29,0.205569,0.206802,0.202849,0.188226,0.196554
2013-12-02,0.209915,0.201402,0.207688,0.187010,0.193985


In [169]:
date_slider = w.SelectionSlider(
    options=date_idx,
    value=date_idx[0],
    description='Date',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

In [170]:
fig_weight = bplt.figure()
plt_weight = bplt.bar(weight.index.to_list(), weight.values)

In [171]:
def update_plt_weight(*arg, **kwargs):
    t = date_slider.value
    weight = data.loc[t]
    with fig_weight.hold_sync():
        plt_weight.y = weight.values

In [172]:
date_slider.observe(update_plt_weight)

In [174]:
w.VBox([date_slider, fig_weight])